# 温度変化dfの前処理

## おまじないパート

In [ ]:
import sys
from pathlib import Path

# 現在のノートブックのパスを取得
notebook_dir = Path().resolve() 

# プロジェクトルートディレクトリ（notebooksディレクトリの親ディレクトリ）を取得
# プロジェクトルートは2階層上です
project_root = notebook_dir.parent.parent

# プロジェクトルートをPythonの検索パスに追加
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [ ]:
from groom import DataCleaner, save_to_parquet, load_from_parquet


In [ ]:
self = DataCleaner()

In [ ]:
import pandas as pd

## 前処理

### 読み込み

In [ ]:
raw_temperature_df = pd.read_csv(project_root / "data" / "t=0_1_1.csv")
print(raw_temperature_df.head())

### 日時表記の整理

In [ ]:
# --- 実行 ---
raw_temperature_df = self.process_datetime_columns(raw_temperature_df, year=2025)
print(raw_temperature_df.head())

### sample_idの付与

In [ ]:
labeled_temperature_df = self.add_sampling_id(raw_temperature_df)
print(labeled_temperature_df.head(20))

### 不要な列の削除

- "facehigh"
- "facelow" 
- "facehigh.1" 
- "facelow.1" 
- "luminocity"

In [ ]:
dropped_temperature_df = self.drop_unnecessary_columns(raw_temperature_df)
print(dropped_temperature_df.head())